In [191]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

### 1. Load file

In [192]:
import os
import pandas as pd

pd.options.display.max_colwidth = 100000000
# C:/Users/doudi/OneDrive/Documents/TMU-GIDS/機器學習與深度學習/Final term
os.chdir('C:/Users/doudi/OneDrive/Documents/TMU-GIDS/機器學習與深度學習/Final term')
path = 'reviews.csv'
df = pd.read_csv(path, encoding='utf-8')
df.head()

,game_id,useful_num,funny_num,user_name,games_owned,reviews_written,recommended,hours_played,review_date,text
0,1007350.0,0,0,76561198155658553,60,3,1,5.6,"1 December, 2019",各種更新不一樣的角色!!!挺有趣
1,1007350.0,0,0,76561198153548993,77,2,1,0.4,"1 December, 2019",蚌
2,1007350.0,0,0,cf40412cf,31,4,1,0.6,"30 November, 2019",期待神人完成劇本
3,1007350.0,0,0,76561198229928252,75,9,1,0.3,"28 November, 2019","準備好你的右手,無須多言"
4,1007350.0,0,0,hucktifs,260,19,1,0.2,"28 November, 2019",可惜沒多少作品


### 2. Preprocessing

In [194]:
# stop word
stw=[]
with open('C:/Users/doudi/Downloads/chineses_stop_word.txt', 'r') as file:
    for data in file.readlines():
        data = data.strip()
        stw.append(data)
        
import re
import jieba
# clean out the punctuation and english words
# Chinese charater only!!!
def clean(file):
    data = []
    for l in file:
        r = re.compile(u'[^\u4e00-\u9fa5]')
        l = r.sub(r'',l)
        data.append(l)
    return data
# segmentation 
def segment(data):
    c = []
    for s in data:
        seg = jieba.cut(s, cut_all=False)
        seg_result = list(filter(lambda a: a not in stw, seg))
        c.append(seg_result)
    return c
corpus = clean(df['text'])

> sklearn TFIDF不用斷詞

### 3. Baseline model

In [198]:
X = list(df['text'])
Y = list(df['recommended'])


# Cross_cal
def baseline_model(train, test):
    from sklearn.naive_bayes import BernoulliNB, MultinomialNB
    from sklearn import svm
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    
    from sklearn.model_selection import cross_val_score
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    tfidf_vectorizer = TfidfVectorizer()
    
    Models = ['Bernoulli NB','Multinomial NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree','XG Boost']
    function = [BernoulliNB(),MultinomialNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier(),
                XGBClassifier()]
    perform = []
    
    for i in range(len(function)):
        model = function[i]
        performance = cross_val_score(model, tfidf_vectorizer.fit_transform(train), test, cv=10,
                                      scoring='accuracy')
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")
        print("avg-accuracy: {0}".format(performance.mean()))
        print('\n')
        perform.append(performance.mean())

    result_table = pd.DataFrame({"Models":Models,"Result":perform})
    return result_table

result = baseline_model(X, Y)
result

====  BernoulliNB  ====
avg-accuracy: 0.7905440450015293


====  MultinomialNB  ====
avg-accuracy: 0.8210084803471778


====  SVC  ====
avg-accuracy: 0.8260900521112863


====  LogisticRegression  ====
avg-accuracy: 0.8227066056681844




C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default

====  RandomForestClassifier  ====
avg-accuracy: 0.825544304692324


====  KNeighborsClassifier  ====
avg-accuracy: 0.8193103189580052


====  DecisionTreeClassifier  ====
avg-accuracy: 0.8244142890531674


====  XGBClassifier  ====
avg-accuracy: 0.8226938375374389




,Models,Result
0,Bernoulli NB,0.790544
1,Multinomial NB,0.821008
2,Svm (linear),0.826090
3,Logistic Regression,0.822707
4,Random Forest,0.825544
5,kNN,0.819310
6,Decision Tree,0.824414
7,XG Boost,0.822694
